<a href="https://colab.research.google.com/github/RudraxDave/AmericanSignLanguage_Reader/blob/main/asl_reader_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [1]:
! mkdir ~/.kaggle

In [2]:
! cp kaggle.json ~/.kaggle/

In [3]:
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
! kaggle datasets download grassknoted/asl-alphabet

 99% 1.01G/1.03G [00:04<00:00, 249MB/s]
100% 1.03G/1.03G [00:04<00:00, 221MB/s]


In [ ]:
! unzip /content/asl-alphabet.zip

Importing Libraries

In [6]:
#importing libraries

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import datasets, transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import random_split,DataLoader
import torch.optim as optim



Loading Datasets

In [7]:
# setting file path
# train_filepath = "../input/asl-alphabet/asl_alphabet_train/asl_alphabet_train"
# test_filepath = "../input/asl-alphabet/asl_alphabet_test"

# setting file path

train_filepath = "/content/asl_alphabet_train/asl_alphabet_train"
test_filepath = "/content/asl_alphabet_test"

In [8]:
# defining transforms
train_transforms = transforms.Compose([
    transforms.ToTensor()
])

In [9]:
#Data Loading 
train_dataset = datasets.ImageFolder(train_filepath, transform=train_transforms)
print("Train Dataset Info:\n")
train_dataset

Train Dataset Info:



Dataset ImageFolder
    Number of datapoints: 87000
    Root location: /content/asl_alphabet_train/asl_alphabet_train
    StandardTransform
Transform: Compose(
               ToTensor()
           )

#### DataLoader
DataLoader takes a dataset (such as you would get from ImageFolder) and returns batches of images and the corresponding labels.

In [11]:

train_dataloader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=100, 
                                               shuffle=True,
                                               num_workers=4)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [12]:
train_label = train_dataloader.dataset.classes
print("Labels :\t",train_label,"\nLength of the label list :",len(train_label))

Labels :	 ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space'] 
Length of the label list : 29


In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# data, labels = data.to(device), labels.to(device)

In [16]:
class NN3(nn.Module):
    def __init__(self):
        super(NN3,self).__init__()
        self.NN3 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size=3,stride=1,padding='same'),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride=2),
            
            nn.Conv2d(64,128,kernel_size=3,stride=1,padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride=2),
            
            nn.Conv2d(128,256,kernel_size=3,stride=1,padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride=2)
        )

        self.predict = nn.Sequential(
            nn.Linear(256*25*25,200),
            nn.Dropout(0.5),
            nn.BatchNorm1d(200),
            nn.Dropout(0.2),
            nn.Linear(200,29),
            nn.Softmax(dim=1)
        )
    
    def forward(self,x):
        x = self.NN3(x)
        x = x.view(-1,256*25*25)
        return self.predict(x) 

In [17]:
model3 = NN3()

if torch.cuda.is_available():
    model3.cuda()
    
model3

NN3(
  (NN3): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=(2, 2), stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (predict): Sequential(
    (0): Linear(in_features=160000, out_features=200, bias=True)
    (1): Dropout(p=0.5, inplace=False)
    (2): BatchNorm1d(200, eps=1e-0

In [18]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model3.parameters(), lr= 0.001)

In [19]:
num_epochs = 5
for epoch in range(num_epochs):
    correct = 0
    for batch_idx , (data,labels) in enumerate(train_dataloader):
        data = data.to(device=device)
        labels = labels.to(device= device)
        data, labels = data.cuda(), labels.cuda()
        
        #forward
        scores = model3(data)
        loss = criterion(scores, labels)

        #backprop
        optimizer.zero_grad()
        loss.backward()

        #gradient descent
        optimizer.step()

        #performance
        prediction = torch.max(scores,1)[1]
    
        correct += (prediction == labels).cpu().sum().numpy()

    print("Epoch :",epoch,"\tLoss :",loss.data,"\tAccuracy :",(correct/len(train_dataloader.dataset))*100)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch : 0 	Loss : tensor(2.5452, device='cuda:0') 	Accuracy : 64.40919540229885
Epoch : 1 	Loss : tensor(2.4488, device='cuda:0') 	Accuracy : 94.60689655172413
Epoch : 2 	Loss : tensor(2.4361, device='cuda:0') 	Accuracy : 98.16091954022988
Epoch : 3 	Loss : tensor(2.4333, device='cuda:0') 	Accuracy : 99.02873563218391
Epoch : 4 	Loss : tensor(2.4280, device='cuda:0') 	Accuracy : 99.25402298850575
